In [ ]:
import pandas as pd
import numpy as np
import scanpy as sc

import matplotlib.pyplot as plt
import seaborn as sns

from anticor_features.anticor_features import get_anti_cor_genes
import topo as tp
from topo.tpgraph.intrinsic_dim import IntrinsicDim

In [ ]:
import datetime
print(datetime.datetime.now())

2025-02-28 12:05:34.750400


In [ ]:
metadata = pd.read_csv('./data/ciona_larva/ciona10stage.meta.upload.new.MSTRG.txt', sep='\t', index_col=0, skiprows=2, header=None)
metadata.columns = ['nGene', 'nUMI', 'orig.ident', 'percent.mito', 'stage', 'batch']

In [ ]:
metadata.shape

(137841, 6)

In [ ]:
keep = metadata['stage'] == 'j.larva'
metadata = metadata[keep].copy()

In [ ]:
metadata['batch'].value_counts()

batch
b16    13740
b15     9194
b14     7748
Name: count, dtype: int64

In [ ]:
# lv1 = sc.read_10x_h5('data/ciona_larva/GSE131155_RAW/GSM3764784_larva1_raw_gene_bc_matrices_h5.h5')
# lv1.obs.index = 'lv1'
# lv2 = sc.read_10x_h5('data/ciona_larva/GSE131155_RAW/GSM3764785_larva2_raw_gene_bc_matrices_h5.h5')
# lv2.obs.index = 'lv2'
lv3 = sc.read_10x_h5('data/ciona_larva/GSE131155_RAW/GSM3764786_larva3_raw_gene_bc_matrices_h5.h5')
# lv3.obs.index = 'lv3'

In [ ]:
keep = metadata.index.str.contains("lv.4")
test = metadata[keep]
test.index = test.index.str.split('_').str[1] + '-1'

In [ ]:
test

,nGene,nUMI,orig.ident,percent.mito,stage,batch
0,,,,,,
AATCGGTAGAGTACCG-1,1068,5509,lv.4,0.400980,j.larva,b15
ACAGCTACAGACGCAA-1,1085,8740,lv.4,0.564416,j.larva,b15
ATCACGACATCAGTAC-1,1116,9871,lv.4,0.551773,j.larva,b15
CCAATCCAGAATCTCC-1,1008,7061,lv.4,0.510198,j.larva,b15
CCACGGACACGAAATA-1,1062,10398,lv.4,0.339521,j.larva,b15
...,...,...,...,...,...,...
lv.4-1,1397,4067,lv.4,0.023113,j.larva,b14
lv.4-1,2570,7587,lv.4,0.025438,j.larva,b14
lv.4-1,3090,12666,lv.4,0.028583,j.larva,b14


In [ ]:
np.intersect1d(test.index, lv2.obs.index).shape

(4749,)

In [ ]:
np.intersect1d(test.index, lv1.obs.index).shape

(4060,)

In [ ]:
lv2 = sc.read_10x_h5('data/ciona_larva/GSE131155_RAW/GSM3764785_larva2_raw_gene_bc_matrices_h5.h5')
lv2.obs.index = 'lv.2_' + lv2.obs.index.str.replace('-1', '')

In [ ]:
lv1 = sc.read_10x_h5('data/ciona_larva/GSE131155_RAW/GSM3764784_larva1_raw_gene_bc_matrices_h5.h5')

In [ ]:
lv1.var.index

Index(['KH2012:KH.C1.286', 'KH2012:KH.C1.560', 'KH2012:KH.C1.907',
       'KH2012:KH.C1.1180', 'KH2012:KH.C1.526', 'KH2012:KH.C1.183',
       'KH2012:KH.C1.172', 'KH2012:KH.C1.969', 'KH2012:KH.C1.655',
       'KH2012:KH.C1.962',
       ...
       'ENSCING00000025220', 'ENSCING00000025221', 'ENSCING00000025222',
       'ENSCING00000025223', 'ENSCING00000025224', 'ENSCING00000025225',
       'ENSCING00000025226', 'ENSCING00000025227', 'ENSCING00000025228',
       'ENSCING00000025229'],
      dtype='object', length=15269)

In [ ]:
keep = metadata['orig.ident'] == 'lv.1'
metadata[keep].index

Index(['lv.1_AAACCTGAGCACGCCT-1', 'lv.1_AACTCAGAGACTTGAA-1',
       'lv.1_AAGGTTCTCGAACGGA-1', 'lv.1_ACCCACTAGCCACTAT-1',
       'lv.1_ACGCCGAAGGGCTTCC-1', 'lv.1_ACGGCCAGTAGCGTAG-1',
       'lv.1_ACTTTCAAGTGTCCAT-1', 'lv.1_AGAGCGAGTATCAGTC-1',
       'lv.1_AGCTTGAGTAACGCGA-1', 'lv.1_AGGCCGTGTATAGGGC-1',
       ...
       'MSTRG_lv.1_TTGGAACCAGACGCAA-2', 'MSTRG_lv.1_TTGGCAATCGTAGGTT-2',
       'MSTRG_lv.1_TTTATGCAGTCGTACT-2', 'MSTRG_lv.1_TTTATGCCATGTCGAT-2',
       'MSTRG_lv.1_TTTCCTCGTAGAGGAA-2', 'MSTRG_lv.1_TTTCCTCTCCAGAGGA-2',
       'MSTRG_lv.1_TTTGCGCTCCTTGACC-2', 'MSTRG_lv.1_TTTGCGCTCTTAGCCC-2',
       'MSTRG_lv.1_TTTGGTTGTCCTCCAT-2', 'MSTRG_lv.1_TTTGTCACAGATCCAT-2'],
      dtype='object', name=0, length=13740)

In [ ]:
metadata.index.str.split('_').str[0].unique()

Index(['lv.3', 'lv.4', 'lv.1', 'MSTRG'], dtype='object', name=0)

In [ ]:
np.intersect1d(metadata.index, lv1.obs.index)

array([], dtype=object)

In [ ]:
raw

In [ ]:
raw.X.max()